In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
# Parameters
n = 1000 # only for computing transition matrix, will be rescaled later.
a = 10
E = 1
c1 = 2
c2 = 1
delta = 0.9
eta = 0.1
nu_1 = 1
nu_2 = 0

# Equilibrium expressions, as derived in the pdf
p = a - (17 / 3)
V1 = E / delta

# Iteration function
def single_update(V2):
    # Calculate investment levels with proper handling of negative arguments (equations 13 and 14)
    x1_arg = delta * (((2 * eta) - 1) * V1 + (1 - eta) * V2)
    x1 = max(0, np.sqrt(max(0, x1_arg)) - 1)

    x2_arg = delta * eta * (V2 - V1)
    x2 = max(0, np.sqrt(max(0, x2_arg)) - 1)

    # Calculate transition probabilities
    rho1 = x1 / (1 + x1)
    rho2 = x2 / (1 + x2)

    # Find stationary distribution using constrained optimization (equations 7 - 9)
    def objective(x):
        f1, f2, phi = x
        f0 = n - f1 - f2

        eq1 = (f0 * (1 - phi)) + (f1 * (1 - rho1) * eta) - f0
        eq2 = (f0 * phi) + (f1 * (1 - (1 - rho1) * eta - rho1 * (1 - eta))) + (f2 * (1 - rho2) * eta) - f1
        eq3 = (f1 * rho1 * (1 - eta)) + (f2 * (1 - (1 - rho2) * eta)) - f2

        return eq1**2 + eq2**2 + eq3**2

    # Constraints to ensure valid distribution
    constraints = [
        {'type': 'ineq', 'fun': lambda x: x[0]},          # f1 >= 0
        {'type': 'ineq', 'fun': lambda x: x[1]},          # f2 >= 0
        {'type': 'ineq', 'fun': lambda x: n - x[0] - x[1]}, # f0 >= 0
        {'type': 'ineq', 'fun': lambda x: x[2]},          # phi >= 0
        {'type': 'ineq', 'fun': lambda x: 1 - x[2]}       # phi <= 1
    ]

    # Try multiple starting points to increase chances of finding valid solution (for numerical accuracy)
    best_result = None
    best_obj = float('inf')

    for f1_start, f2_start in [(n/3, n/3), (n/2, n/4), (n/4, n/2), (n-1, 1), (1, n-1)]:
        result = minimize(objective, [f1_start, f2_start, 0.5],
                         constraints = constraints, method = 'SLSQP')

        if result.fun < best_obj:
            best_obj = result.fun
            best_result = result

    f1, f2, phi = best_result.x
    f0 = (n - f1 - f2) * (10 / n)

    # Add small regularization to prevent division by zero and rescale to n = 10 for the question at hand
    f1 = max(0.001, f1) * (10 / n)
    f2 = max(0.001, f2) * (10 / n)

    # Calculate profits
    pi1 = (p - c1) * (7 / (3 * f1)) - x1
    pi2 = (p - c2) * (10 / (3 * f2)) - x2

    # Compute the implied value functions
    V2_new = pi2 + delta * (((1 - rho2) * eta) * V1 + (1 - ((1 - rho2) * eta)) * V2)

    return {
        'V2_new': V2_new,
        'x1': x1,
        'x2': x2,
        'f0': f0,
        'f1': f1,
        'f2': f2
    }

V2_values = np.linspace(7, 10, 3001)

# Implementation (takes about 30 seconds to answer)
results = []
for V2 in V2_values: #
    if abs(single_update(V2)['V2_new'] - V2) < 0.0001:
      print(V2, single_update(V2))
      break

7.5969999999999995 {'V2_new': 7.596970179144673, 'x1': 1.3137782953429222, 'x2': 0, 'f0': 0.11062589593407211, 'f1': 1.6184870790355887, 'f2': 8.270887025030339}
